In [1]:
!pip install transformers
!pip install datasets
!pip install sentencepiece
!pip install rouge_score
!pip install simplet5 -q

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=60802403acbd578f58c55ee0eae6b86dce53bcd6d73398268ac826a2af7979ae
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda 23.7.3 requires setuptools>=60.0.0, but you have setuptools 59.5.0 which is incompatible.
opentelemetry-api 1.18.0 requires importlib-metadata~=6.0.0, but you have importlib-metadata 6.7.0 which is incompatible.
pymc3 3.11.5 requires numpy<1.22.2,>=1.15.0, but you have numpy 1.23.5 which is incompatible.
pymc3 3.11.5 requires scipy<1.8.0,>=1.7.3, but you have scipy 1.11.2 which is incompatible.


In [2]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config
from simplet5 import SimpleT5
from tabulate import tabulate
from datasets import Dataset
import nltk
from datetime import datetime
import numpy as np
import pickle
import datasets

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:

import os
os.environ["WANDB_DISABLED"] = "true"

In [4]:
Train = open("/kaggle/input/rankar-data/Ranked-data/Train-cross-encoder.pickle","rb")
Train_data = pickle.load(Train)

In [5]:
Val = open("/kaggle/input/rankar-data/Ranked-data/Val-cross-encoder.pickle","rb")
Val_data = pickle.load(Val)

In [6]:
Test = open("/kaggle/input/rankar-data/Ranked-data/Test-cross-encoder.pickle","rb")
Test_data = pickle.load(Test)

In [7]:
import pandas as pd
import datasets

# Assuming Train_data, Val_data, and Test_data are lists of dictionaries
train_df = pd.DataFrame(Train_data)
val_df = pd.DataFrame(Val_data)
test_df = pd.DataFrame(Test_data)


In [8]:
# Convert Pandas DataFrames to datasets
train_data_txt = datasets.Dataset.from_pandas(train_df)
val_data_txt = datasets.Dataset.from_pandas(val_df)
test_data_txt = datasets.Dataset.from_pandas(test_df)

In [9]:
def flatten(example):
  answers = example["rankedanswers"]
#  print(answers[0])
  allrankedanswers = ""
  for answer in answers:
    allrankedanswers += answer + " "
    
  return {
      
      "question": example["question"],
      "allrankedanswers": allrankedanswers,
      "firstsummary": example["firstsummary"],
      "question+allrankedanswers": example["question"] + " " + allrankedanswers
      }

In [10]:
train_data_txt = train_data_txt.map(flatten, remove_columns=['rankedanswers'])
val_data_txt = val_data_txt.map(flatten, remove_columns=['rankedanswers'])
test_data_txt = test_data_txt.map(flatten, remove_columns=['rankedanswers'])

  0%|          | 0/2783 [00:00<?, ?ex/s]

  0%|          | 0/500 [00:00<?, ?ex/s]

  0%|          | 0/1000 [00:00<?, ?ex/s]

In [11]:
len(val_data_txt)

500

In [12]:

val_data_txt[0]


{'question': 'Assume that someone (friend, family) asks you to format his/her CV. They typically might want to apply to a job where text processing etc. is not one of the key requirements and they ask you to do it in order to have a nice result and/or to win time, although they could probably do it themselves in Word.  Can it be negative for them? Imagine for instance that the interviewer asks them how they made such a nice CV.',
 'firstsummary': "Saving seeds for next year is not difficult as long as you aren't using hybrid varieties. Some plants are hard to get seeds from if you live in a cold climate or require special techniques.",
 'allrankedanswers': 'Is it acceptable to format a cv for someone else? Nobody knows or cares who wrote or formatted your CV/resume. Having someone else format it for you is likely to lead them to a false conclusion. If the C.V. or job description describes the person as being good at design, formatting documents, or experienced with LaTeX, then the pers

In [13]:
# Model-Bart trained on X-Sum

model_name = "sshleifer/distilbart-xsum-12-3"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/683M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [14]:
# Model T-5
#model = T5ForConditionalGeneration.from_pretrained('t5-small')
#tokenizer = T5Tokenizer.from_pretrained('t5-base')


In [27]:
encoder_max_length = 512
decoder_max_length = 100
input = "allrankedanswers"
output = "firstsummary"

train_data_txt.column_names

training_args = Seq2SeqTrainingArguments(
    output_dir="results",
    num_train_epochs=5,  # demo
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=2,  # demo
    per_device_eval_batch_size=2,
    # learning_rate=3e-05,
    warmup_steps=100,
    weight_decay=0.1,
    label_smoothing_factor=0.1,
    predict_with_generate=True,
    logging_dir="logs",
    logging_steps=200,    ## it take lot of output space  so i incresead logging step 50 to 200
    save_total_limit=1,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [28]:
def batch_tokenize_preprocess(batch, tokenizer, max_source_length, max_target_length):
    source, target = batch[input], batch[output]
    source_tokenized = tokenizer(
        source, padding="max_length", truncation=True, max_length=max_source_length
    )
    target_tokenized = tokenizer(
        target, padding="max_length", truncation=True, max_length=max_target_length
    )

    batch = {k: v for k, v in source_tokenized.items()}
    # Ignore padding in the loss
    batch["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in l]
        for l in target_tokenized["input_ids"]
    ]
    return batch

In [29]:
train_data = train_data_txt.map(
    lambda batch: batch_tokenize_preprocess(
        batch, tokenizer, encoder_max_length, decoder_max_length
    ),
    batched=True,
    remove_columns=train_data_txt.column_names,
)

  0%|          | 0/3 [00:00<?, ?ba/s]

In [30]:
val_data = val_data_txt.map(
    lambda batch: batch_tokenize_preprocess(
        batch, tokenizer, encoder_max_length, decoder_max_length
    ),
    batched=True,
    remove_columns=val_data_txt.column_names,
)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [31]:
test_data = test_data_txt.map(
    lambda batch: batch_tokenize_preprocess(
        batch, tokenizer, encoder_max_length, decoder_max_length
    ),
    batched=True,
    remove_columns=test_data_txt.column_names,
)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [32]:
#METRICS
nltk.download("punkt", quiet=True)

metric = datasets.load_metric("rouge")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

    return preds, labels

In [33]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    # Extract a few results from ROUGE
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    prediction_lens = [
        np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds
    ]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result


In [34]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [35]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_data,
    eval_dataset=val_data,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [36]:

#EVALUATE BEFORE FINE TUNING
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 500
  Batch size = 4


{'eval_loss': 6.488889694213867,
 'eval_rouge1': 12.6251,
 'eval_rouge2': 1.3119,
 'eval_rougeL': 10.3298,
 'eval_rougeLsum': 11.0816,
 'eval_gen_len': 14.904,
 'eval_runtime': 92.0145,
 'eval_samples_per_second': 5.434,
 'eval_steps_per_second': 1.358}

In [37]:
#TRAIN THE MODEL
trainer.train()

***** Running training *****
  Num examples = 2783
  Num Epochs = 5
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 3480


Step,Training Loss
200,3.191000
400,1.906100
600,1.745900
800,1.677700
1000,1.654400
1200,1.654500
1400,1.616900
1600,1.605100
1800,1.609500
2000,1.596800


Saving model checkpoint to results/checkpoint-500
Configuration saved in results/checkpoint-500/config.json
Model weights saved in results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in results/checkpoint-500/tokenizer_config.json
Special tokens file saved in results/checkpoint-500/special_tokens_map.json
Saving model checkpoint to results/checkpoint-1000
Configuration saved in results/checkpoint-1000/config.json
Model weights saved in results/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in results/checkpoint-1000/tokenizer_config.json
Special tokens file saved in results/checkpoint-1000/special_tokens_map.json
Deleting older checkpoint [results/checkpoint-500] due to args.save_total_limit
Saving model checkpoint to results/checkpoint-1500
Configuration saved in results/checkpoint-1500/config.json
Model weights saved in results/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in results/checkpoint-1500/tokenizer_config.json
Special tokens 

TrainOutput(global_step=3480, training_loss=1.7170055805951698, metrics={'train_runtime': 2498.799, 'train_samples_per_second': 5.569, 'train_steps_per_second': 1.393, 'total_flos': 8615582367744000.0, 'train_loss': 1.7170055805951698, 'epoch': 5.0})

In [38]:

#EVALUATE AFTER FINE TUNING
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 500
  Batch size = 4
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 7.032856464385986,
 'eval_rouge1': 14.1805,
 'eval_rouge2': 1.041,
 'eval_rougeL': 10.2796,
 'eval_rougeLsum': 12.7389,
 'eval_gen_len': 38.228,
 'eval_runtime': 127.0731,
 'eval_samples_per_second': 3.935,
 'eval_steps_per_second': 0.984,
 'epoch': 5.0}

In [40]:
#Generate summaries from the fine-tuned model and compare them with those generated from the original, pre-trained one.
def generate_summary(test_samples, model):
    inputs = tokenizer(
        test_samples["allrankedanswers"],
        padding="max_length",
        truncation=True,
        max_length=encoder_max_length,
        return_tensors="pt",
    )
    input_ids = inputs.input_ids.to(model.device)
    attention_mask = inputs.attention_mask.to(model.device)
    outputs = model.generate(input_ids, attention_mask=attention_mask)
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return outputs, output_str

In [41]:
model_before_tuning = AutoModelForSeq2SeqLM.from_pretrained(model_name)
#model_before_tuning = T5ForConditionalGeneration.from_pretrained('t5-small')
test_samples = val_data_txt.select(range(5))
model_after_training = AutoModelForSeq2SeqLM.from_pretrained("sshleifer/distilbart-xsum-12-3")
summaries_before_tuning = generate_summary(test_samples, model_before_tuning)[1]
summaries_after_tuning = generate_summary(test_samples, model)[1]
summaries_after_training = generate_summary(test_samples, model_after_training)[1]

loading configuration file https://huggingface.co/sshleifer/distilbart-xsum-12-3/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/4a0f7fb35f1504b6e865136124e3781fb488792aa105a84a991a3145a027791f.10ebe969457e130b9da526e7994b6191d3765d1d01ac6abc2eb20bb8adcbd4e0
Model config BartConfig {
  "_name_or_path": "sshleifer/distilbart-xsum-12-3",
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 3,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_t

In [45]:
for i in range(5):
  print(val_data_txt['firstsummary'][i])

# print(summaries_before_tuning)

# print(summaries_after_tuning)

Saving seeds for next year is not difficult as long as you aren't using hybrid varieties. Some plants are hard to get seeds from if you live in a cold climate or require special techniques.
They won't have much of an effect on your plants however they do feed aphids.
The cartridges don't fit smoothly into the slot and the box is not official. They also might not work on newer versions of the console. The cartridge could also be bigger or a different shade of grey.
Nobody will check this valuation unless you are audited, so you just need to be able to justify the amount. You can look for similar items for sale on eBay. The IRS isn't going to quibble if your numbers are reasonable.
They are quite useful. Some high-level plays build extra Fortresses in Yellow Mine Expansions.


In [43]:
print(summaries_before_tuning)

[" On the eve of a job interview, I've been asking someone to proofread your resume.", ' In our series of letters from African-American journalists, film-maker and columnist Farai Sevenzo considers whether to send a resume by mail or courier.', " An employee's notice period is not ethical, and it's not ethical.", ' There is a kind of "trick" question to see if one jumps into giving a technical answer to the question.', ' In our series of letters from African journalists, film-maker and columnist Farai Sevenzo looks at the challenges faced by people living in your office.']


In [44]:
print(summaries_after_tuning)

['It is not necessary to bring up this information if you are not asked about it but if they ask you about it then you should definitely answer honestly. You could bring it up in the interview to save the company being surprised by it.', "You should not assume anything and make sure you check your contract. However, if you haven't been let go already then there is a good chance you have passed your probation.", 'You should inform the customer that they should call your work number and leave a message. You could even block them from calling you on that number. Discuss the issue with your management.', "The best way is to immediately correct them so they don't make the same mistake again. You could also introduce yourself with the right title at the start of the call. If it is someone you probably won't speak to again, then just ignore it.", "Don't worry about it. Think about your interview technique for future applications. Don't consider offers from companies that have already done thi